In [1]:
from src.constants.constants import *
from src.feature_extraction.extract_features import *
from src.utils.helper import *
from src.models.evaluate import *
from src.models.train import *
from src.preprocessing.audio_cleaning import *
# from src.feature_extraction.save_features import *
import warnings

# Silence LightGBM warnings
warnings.filterwarnings("ignore")


In [2]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.mixture import GaussianMixture



In [4]:
data = pd.read_csv('csv_files/data.csv')
features = pd.read_csv('csv_files/features_40_MFCC.csv')
df = pd.merge(features, 
              data.drop(columns=['client_id', 'sentence', 'up_votes', 'down_votes', 'accent']), 
              left_on='voice_id', right_on='path', how='inner')
df.drop(columns=['path', 'Unnamed: 0'], inplace=True)
df.head()

,mean_mfcc_0,mean_mfcc_1,mean_mfcc_2,mean_mfcc_3,mean_mfcc_4,mean_mfcc_5,mean_mfcc_6,mean_mfcc_7,mean_mfcc_8,mean_mfcc_9,...,std_delta_mfcc_35,std_delta_mfcc_36,std_delta_mfcc_37,std_delta_mfcc_38,std_delta_mfcc_39,pitch,voice_id,age,gender,label
0,-403.22006,99.76226,20.534065,41.544975,-7.227066,31.042414,-2.121207,-0.940516,2.747488,7.995158,...,1.270415,0.916675,1.036257,0.922543,0.850381,1656.88460,common_voice_en_181169.mp3,fifties,male,2
1,-223.81348,157.47336,11.694227,66.153180,-23.514465,0.811838,-19.525723,-8.694965,-6.325938,-10.528963,...,1.447682,0.839711,1.264348,1.171347,1.692225,1285.26820,common_voice_en_122004.mp3,twenties,male,0
2,-415.86557,126.01485,-36.185616,48.158478,-13.574215,-18.600720,-39.063217,-24.592718,-17.863024,-13.136163,...,1.995541,1.724953,1.857788,1.803757,2.830431,1702.50980,common_voice_en_10208833.mp3,fifties,female,3
3,-423.24472,198.54523,12.053857,62.119160,17.754673,-21.133663,6.490955,-1.768311,-28.847279,-15.727254,...,1.094712,2.345557,2.794774,2.051888,1.940512,820.08923,common_voice_en_18618243.mp3,twenties,male,0
4,-223.69318,74.87990,9.623554,114.356290,-12.664084,-20.089724,-38.216280,-0.664027,-42.319390,-13.502878,...,0.912976,1.264993,1.243301,0.944270,1.173495,1860.53880,common_voice_en_18075517.mp3,twenties,male,0


In [11]:
X = df.drop(columns=['label', 'gender', 'age', 'voice_id'])
y = df['gender']
le = LabelEncoder()
y = le.fit_transform(y)
X_val_train, X_test, y_val_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_val_train, y_val_train, test_size=0.18, random_state=42, stratify=y_val_train)

In [81]:
# model = load_model('src/saved_models/best_xgb_model.pkl')

In [13]:
%%time 
pipeline = get_pipeline(name='knn')
model = train_model(pipeline, X_train, y_train)
model

CPU times: user 243 ms, sys: 105 ms, total: 347 ms
Wall time: 424 ms


Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [82]:
%%time
evaluate_multi(model, X_train, y_train, type = 0)



nTRAINING TESTING RESULTS: 
CONFUSION MATRIX:
[[  246 13412     4 10848]
 [84116    96 11235    36]
 [    0     0     0     0]
 [    0     0     0     0]]
ACCURACY SCORE:
0.0029
CPU times: user 6.7 s, sys: 57.8 ms, total: 6.75 s
Wall time: 723 ms


In [15]:
%%time
evaluate_multi(model, X_val, y_val, type = 1)


\VALIDATION RESULTS: 
CONFUSION MATRIX:
[[ 5143   238]
 [  368 20592]]
ACCURACY SCORE:
0.9770
CPU times: user 1min 17s, sys: 2.61 s, total: 1min 20s
Wall time: 9.94 s


In [14]:
%%time
evaluate_multi(model, X_test, y_test, type = 2)



TESTING RESULTS: 
CONFUSION MATRIX:
[[ 5039   236]
 [  366 20183]]
ACCURACY SCORE:
0.9767
CPU times: user 1min 10s, sys: 2.38 s, total: 1min 13s
Wall time: 9.02 s


In [29]:
# import joblib
# joblib.dump(model, 'src/selected_model/knn_base_40_.pkl')

['src/selected_model/knn_base_40_.pkl']

In [16]:
model = load_model('src/selected_model/knn_base_40.pkl')